# Day 2 - Intcode interpreter

* https://adventofcode.com/2019/day/2

We have a computer again! We've seen this before in 2017 ([day 18](../2017/Day%2018.ipynb), [day 23](../2017/Day%2023.ipynb)), and 2018 ([day 16](../2018/Day%2016.ipynb), [day 19](../2018/Day%2019.ipynb) and [day 21](../2018/Day%2021.ipynb)).

Now we have opcodes with a variable number of operands (called *positions* here); `1` and `2` each have 2 operands and output destination, 99 has none. There are also no registers, all operations take place directly on the memory where our code is also stored, so it can self-modify. Fun!

So we need a CPU with a position counter, memory, and opcode definitions (*instructions*) to call, and the opcodes need access to the memory (to read operand values and write out their result to). Easy peasy.

I'm assuming we'll expand on the instruction set later on, and that we might have instructions with different numbers of operands. So given a function to process the input values and the number of *paramaters* to process, we should be able to produce something readable and reusable.

In [1]:
import aocd
data = aocd.get_data(day=2, year=2019)
memory = list(map(int, data.split(',')))

In [2]:
from __future__ import annotations

import operator
from dataclasses import dataclass
from typing import Callable, List, Mapping, Optional

Memory = List[int]

class Halt(Exception):
    """Signal to end the program"""
    @classmethod
    def halt(cls) -> int:  # yes, because Opcode.f callables always produce ints, right?
        raise cls

@dataclass
class Instruction:
    # the inputs are processed by a function that operates on integers
    # returns integers to store in a destination position
    f: Callable[..., int]
    # An opcode takes N paramaters
    paramater_count: int

    def __call__(self, memory: Memory, *parameters: int) -> None:
        if parameters:
            *inputs, output = parameters
            memory[output] = self.f(*(memory[addr] for addr in inputs))
        else:
            # no parameter count, so just call the function directly, no output expected
            self.f()


class CPU:
    memory: Memory
    pos: int
    opcodes: Mapping[int, Instruction] = {
        1: Instruction(operator.add, 3),
        2: Instruction(operator.mul, 3),
        99: Instruction(Halt.halt, 0),
    }
        
    def reset(self, memory: Memory = None):
        if memory is None:
            memory = []
        self.memory = memory[:]
        self.pos: int = 0
        
    def execute(
        self, memory: Memory,
        noun: Optional[int] = None,
        verb: Optional[int] = None
    ) -> int:
        self.reset(memory)
        memory = self.memory
        if noun is not None:
            memory[1] = noun
        if verb is not None:
            memory[2] = verb
        try:
            while True:
                op = self.opcodes[memory[self.pos]]
                paramcount = op.paramater_count
                parameters = memory[self.pos + 1 : self.pos + 1 + paramcount]
                op(memory, *parameters)
                self.pos += 1 + paramcount
        except Halt:
            return memory[0]

test: Memory = [1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50]
cpu = CPU()
assert cpu.execute(test) == 3500

In [3]:
print('Part 1:', cpu.execute(memory, 12, 2))

Part 1: 3706713


## Part 2

Now we need to find the noun and verb that produce a specific programme output. The text suggests we should just brute-force this, so lets try that first and see how long that takes. Given that we'll only have to search through 10000 different inputs, that's not that big a search space anyway.

In [4]:
from itertools import product
def bruteforce(target: int, memory: Memory) -> int:
    cpu = CPU()
    for noun, verb in product(range(100), repeat=2):
        result = cpu.execute(memory, noun, verb)
        if result == target:
            break
    return 100 * noun + verb

print('Part 2:', bruteforce(19690720, memory))

Part 2: 8609
